In [ ]:
import os
import pandas as pd
import numpy as np
import google

In [ ]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 15.5 MB/s eta 0:00:00


In [ ]:
pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
pip install dash_bootstrap_templates

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 1.9 MB/s eta 0:00:00


In [ ]:
pip install pandas fuzzywuzzy python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.8 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [1]:
sh = gc.open('Season 21 Statistics') # <-- Look ma, no keys!
# extract each tab into a uniquely named dataframe based on the tab

dataframes = {}
for sheet in sh.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

overall = dataframes['Sheet1']

# clean up the df and use the titles in the first row ('FT Made', 'Dunks' etc) the column names
overall = overall.rename(columns=overall.iloc[0])
overall = overall.drop(overall.index[0])

# replace '#DIV/0!' with numpy NaN
overall.replace('#DIV/0!', np.nan, inplace=True)

for column in overall.columns:
  if column == 'Name':
    overall[column] = overall[column].astype(str)
  elif overall[column].astype(str).str.contains('%').any():  # Check if any string in the column contains '%'
    overall[column] = overall[column].str.rstrip('%').astype('float') / 100.0
  else:
    overall[column] = overall[column].astype(float)

overall = overall.round(2)
overall['Name'] = overall['Name'].apply(lambda x: ' '.join(x.split(',')[::-1]))
overall.applymap(lambda x: x.strip() if isinstance(x, str) else x)

NameError: name 'gc' is not defined

In [ ]:
%%time

from fuzzywuzzy import process

sh = gc.open('Season 21 Statistics') # <-- Look ma, no keys!
# extract each tab into a uniquely named dataframe based on the tab

dataframes = {}
for sheet in sh.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

overall = dataframes['Sheet1']

# clean up the df and use the titles in the first row ('FT Made', 'Dunks' etc) the column names
overall = overall.rename(columns=overall.iloc[0])
overall = overall.drop(overall.index[0])

# replace '#DIV/0!' with numpy NaN
overall.replace('#DIV/0!', np.nan, inplace=True)

for column in overall.columns:
  if column == 'Name':
    overall[column] = overall[column].astype(str)
  elif overall[column].astype(str).str.contains('%').any():  # Check if any string in the column contains '%'
    overall[column] = overall[column].str.rstrip('%').astype('float') / 100.0
  else:
    overall[column] = overall[column].astype(float)

overall = overall.round(2)
overall['Name'] = overall['Name'].apply(lambda x: ' '.join(x.split(',')[::-1]))
overall.applymap(lambda x: x.strip() if isinstance(x, str) else x)

s22_draft = gc.open('Season 22 Draft')
dataframes = {}
for sheet in s22_draft.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

dataframes.pop('DRAFT!')

s22_draftable = pd.concat(dataframes, axis=0)
s22_draftable = s22_draftable.rename(columns={0: "Name", 1: "Rating"})
s22_draftable = s22_draftable.reset_index(level=1, drop=True)
s22_draftable['Position'] = s22_draftable.index
s22_draftable = s22_draftable.reset_index(drop=True)
s22_draftable['Name'] = s22_draftable['Name'].apply(lambda x: ' '.join(x.split(',')[::-1]))
s22_draftable.applymap(lambda x: x.strip() if isinstance(x, str) else x)

for column in s22_draftable.columns:
  if column == 'Name':
    s22_draftable[column] = s22_draftable[column].astype(str)
  elif column == 'Position':
    s22_draftable[column] = s22_draftable[column].astype(str)
  else:
    s22_draftable[column] = s22_draftable[column].astype(int)

# merged_df = pd.merge(overall, s22_draftable, on='Name')

# Fuzzy matching and merging
names = overall['Name'].tolist()
choices = s22_draftable['Name'].tolist()
matches = {n: process.extractOne(n, choices) for n in names}  # Find the best match for each name in 'overall'

# Update 'Name' in 'overall' with matched names from 's22_draftable' if the match score is above 90
for name, (matched_name, score) in matches.items():
    if score >= 90:  # assuming a threshold score of 90 for a match
        overall.loc[overall['Name'] == name, 'Name'] = matched_name

# Merging the dataframes on 'Name'
merged_df = pd.merge(overall, s22_draftable, on='Name')
merged_df

CPU times: user 9.83 s, sys: 27.1 ms, total: 9.86 s
Wall time: 19.5 s


,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,EFG%,Fouls,Plus Minus,VORP,Dunks,Minutes,PER,Game Score,Rating,Position
0,Kareem Abdul-Jabbar,29.68,4.92,16.12,1.76,1.68,4.08,5.76,1.44,1.22,...,0.61,2.88,0.28,1.88,2.80,39.52,36.64,28.48,99,C
1,Shareef Abdur-Rahim,2.00,1.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,...,NaN,1.00,-8.00,-2.13,0.00,17.00,8.27,2.90,86,PF
2,Steven Adams,1.29,0.79,2.54,0.38,0.46,0.50,0.96,0.08,4.50,...,1.04,0.54,0.50,0.55,0.13,10.50,12.69,2.55,82,C
3,Bam Adebayo,7.72,0.88,4.84,1.00,0.92,0.36,1.28,0.32,3.13,...,0.74,1.16,-3.16,-0.43,1.68,17.60,23.37,8.22,87,PF
4,Mark Aguirre,3.44,0.08,1.48,0.80,0.12,0.08,0.20,0.36,2.22,...,0.75,0.60,-4.08,-0.59,0.12,13.52,8.01,2.20,88,SF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Zion Williamson,11.28,1.08,3.56,2.72,0.20,0.16,0.36,1.16,2.34,...,0.67,1.32,-8.28,-3.91,2.24,29.92,15.08,9.32,92,PF
322,Kevin Willis,2.67,0.67,3.67,1.00,0.00,0.50,0.50,0.33,3.00,...,0.83,0.33,3.50,1.59,0.00,13.83,12.82,3.55,87,PF
323,James Worthy,10.24,1.24,6.80,2.40,0.80,0.36,1.16,1.08,2.22,...,0.66,1.16,0.64,1.95,1.76,35.48,14.16,10.19,95,SF
324,Ming Yao,13.24,4.00,14.04,2.16,1.24,3.00,4.24,0.84,2.57,...,0.89,3.04,-1.68,0.23,0.88,34.20,28.72,19.00,91,C


In [ ]:
from fuzzywuzzy import process

# Replace a few troublesome typos
# merged_df['Name'].replace('Ming Yao', 'Yao Ming', inplace=True)
# merged_df['Name'].replace('Jaren Jr Jackson', 'Jaren Jackson Jr', inplace=True)

s22_draft = gc.open('Season 22 Draft')
dataframes = {}
for sheet in s22_draft.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

tmp = dataframes['DRAFT!']

# Stripping whitespace and reformatting names for accurate matching
merged_df['Name'] = merged_df['Name'].str.strip()
tmp = tmp.applymap(lambda x: ' '.join(x.strip().split(', ')[::-1]) if isinstance(x, str) else x)

# Flatten the tmp DataFrame to get a list of names
tmp_flattened = tmp.values.flatten()

# Initialize a column for draft status
merged_df['Draft Status'] = 'Available'

# Apply fuzzy matching to determine the most similar names and update 'Draft Status'
for i, row in merged_df.iterrows():
    # Find the best match for the name in the flattened tmp names list
    best_match, score = process.extractOne(row['Name'], tmp_flattened)
    # If the score is high enough, consider it a match and update 'Draft Status'
    if score >= 90:  # assuming a threshold score of 90 for a match
        merged_df.at[i, 'Draft Status'] = 'Drafted'

# Calculate 'Points per Minute' (PPM) assuming 'Points' and 'Minutes' columns are present
merged_df['PPM'] = merged_df['Points'] / merged_df['Minutes']

# Now, merged_df includes the 'Draft Status' column with 'Drafted' or 'Available'
merged_df

,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Plus Minus,VORP,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM
0,Kareem Abdul-Jabbar,29.68,4.92,16.12,1.76,1.68,4.08,5.76,1.44,1.22,...,0.28,1.88,2.80,39.52,36.64,28.48,99,C,Drafted,0.751012
1,Shareef Abdur-Rahim,2.00,1.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,...,-8.00,-2.13,0.00,17.00,8.27,2.90,86,PF,Drafted,0.117647
2,Steven Adams,1.29,0.79,2.54,0.38,0.46,0.50,0.96,0.08,4.50,...,0.50,0.55,0.13,10.50,12.69,2.55,82,C,Drafted,0.122857
3,Bam Adebayo,7.72,0.88,4.84,1.00,0.92,0.36,1.28,0.32,3.13,...,-3.16,-0.43,1.68,17.60,23.37,8.22,87,PF,Drafted,0.438636
4,Mark Aguirre,3.44,0.08,1.48,0.80,0.12,0.08,0.20,0.36,2.22,...,-4.08,-0.59,0.12,13.52,8.01,2.20,88,SF,Available,0.254438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Zion Williamson,11.28,1.08,3.56,2.72,0.20,0.16,0.36,1.16,2.34,...,-8.28,-3.91,2.24,29.92,15.08,9.32,92,PF,Drafted,0.377005
322,Kevin Willis,2.67,0.67,3.67,1.00,0.00,0.50,0.50,0.33,3.00,...,3.50,1.59,0.00,13.83,12.82,3.55,87,PF,Available,0.193059
323,James Worthy,10.24,1.24,6.80,2.40,0.80,0.36,1.16,1.08,2.22,...,0.64,1.95,1.76,35.48,14.16,10.19,95,SF,Drafted,0.288613
324,Ming Yao,13.24,4.00,14.04,2.16,1.24,3.00,4.24,0.84,2.57,...,-1.68,0.23,0.88,34.20,28.72,19.00,91,C,Drafted,0.387135


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Assuming 'merged_df' is the DataFrame resulting from your previous merging steps
# Load 'Season 22 Draft' data
s22_draft = gc.open('Season 22 Draft')
dataframes = {}
for sheet in s22_draft.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

tmp = dataframes['DRAFT!']

# Clean up the tmp DataFrame and use the titles in the first row
tmp = tmp.rename(columns=tmp.iloc[0])
tmp = tmp.drop(tmp.index[0])

# Melting dataframe to long format
melted_df = tmp.melt(id_vars="Round/Team", var_name="Team", value_name="Name")
melted_df.columns = ['Round Picked', 'Team', 'Name']

# Cleaning and preparing the melted_df
melted_df['Round Picked'] = melted_df['Round Picked'].astype(int)
melted_df["Team"] = melted_df["Team"].str.extract(r'(.*) \(')[0]
melted_df = melted_df[melted_df["Name"] != ""]

# Initialize columns in merged_df for Draft Status, Team, and Round Picked
merged_df['Draft Status'] = 'Available'
merged_df['Team'] = 'None'
merged_df['Round Picked'] = 'Unpicked'

# Perform fuzzy matching and merging
for index, merged_row in merged_df.iterrows():
    name_to_match = merged_row['Name']
    best_match, score = process.extractOne(name_to_match, melted_df['Name'].tolist())

    if score >= 90:  # assuming a threshold score of 90 for a match
        matched_row = melted_df[melted_df['Name'] == best_match]

        # Assuming there is only one best match, so use iloc[0] to get the first (and only) match
        merged_df.at[index, 'Draft Status'] = 'Drafted'
        merged_df.at[index, 'Team'] = matched_row['Team'].iloc[0]
        merged_df.at[index, 'Round Picked'] = matched_row['Round Picked'].iloc[0]

# Now merged_df includes the 'Draft Status', 'Team', and 'Round Picked' columns
merged_df


,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM,Team,Round Picked
0,Kareem Abdul-Jabbar,29.68,4.92,16.12,1.76,1.68,4.08,5.76,1.44,1.22,...,2.80,39.52,36.64,28.48,99,C,Drafted,0.751012,Blake B,1
1,Shareef Abdur-Rahim,2.00,1.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,...,0.00,17.00,8.27,2.90,86,PF,Drafted,0.117647,Hayden R,12
2,Steven Adams,1.29,0.79,2.54,0.38,0.46,0.50,0.96,0.08,4.50,...,0.13,10.50,12.69,2.55,82,C,Drafted,0.122857,Elise D,10
3,Bam Adebayo,7.72,0.88,4.84,1.00,0.92,0.36,1.28,0.32,3.13,...,1.68,17.60,23.37,8.22,87,PF,Drafted,0.438636,Brian R,7
4,Mark Aguirre,3.44,0.08,1.48,0.80,0.12,0.08,0.20,0.36,2.22,...,0.12,13.52,8.01,2.20,88,SF,Available,0.254438,None,Unpicked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Zion Williamson,11.28,1.08,3.56,2.72,0.20,0.16,0.36,1.16,2.34,...,2.24,29.92,15.08,9.32,92,PF,Drafted,0.377005,Brian R,3
322,Kevin Willis,2.67,0.67,3.67,1.00,0.00,0.50,0.50,0.33,3.00,...,0.00,13.83,12.82,3.55,87,PF,Available,0.193059,None,Unpicked
323,James Worthy,10.24,1.24,6.80,2.40,0.80,0.36,1.16,1.08,2.22,...,1.76,35.48,14.16,10.19,95,SF,Drafted,0.288613,Garrett H,4
324,Ming Yao,13.24,4.00,14.04,2.16,1.24,3.00,4.24,0.84,2.57,...,0.88,34.20,28.72,19.00,91,C,Drafted,0.387135,Jorge L,3


In [ ]:
import pandas as pd
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import numpy as np

# loads the "darkly" template and sets it as the default
load_figure_template("cyborg")

import plotly.express as px

def create_slider_marks(min_val, max_val, num_marks=20):
    """Create evenly spaced, rounded marks for a slider given min and max values."""
    # Generate evenly spaced numbers between min and max
    step_values = np.linspace(min_val, max_val, num_marks)
    # Create a dictionary of marks (position: label)
    marks = {int(value): '{:.2f}'.format(value) for value in step_values}
    return marks

def preprocess_size_data(size_data):
    """
    This function offsets negative data to ensure all values are positive
    and scales it to be suitable for point sizes in the scatter plot.
    """
    # Offset by the minimum if there are negative values to make all values positive
    min_val = size_data.min()
    if min_val < 0:
        size_data = size_data + abs(min_val)

    # Scale data to a suitable range for point sizes, e.g., 10 to 50
    max_val = size_data.max()
    size_data = 10 + ((size_data - size_data.min()) / max_val) * 40  # Adjust 10 and 50 as necessary
    return size_data


# overall = overall.copy()
overall = merged_df.copy()

# Creating options list for dropdowns
name_options = [{'label': 'All', 'value': 'all'}] + [{'label': name, 'value': name} for name in overall['Name'].unique()]
position_options = [{'label': pos, 'value': pos} for pos in overall['Position'].unique()]
team_options = [{'label': team, 'value': team} for team in overall['Team'].unique()]  # Team dropdown list

# Initialize the Dash app
app = dash.Dash(__name__)
app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])

# Create a list of numeric column names for use in the callback and layout
numeric_cols = [col for col in overall.columns if pd.api.types.is_numeric_dtype(overall[col])]

# Define the layout of the app
app.layout = html.Div([
    # Dropdowns for selecting x and y columns
    dcc.Dropdown(
        id='x-column',
        options=[{'label': col, 'value': col} for col in overall.columns],
        value='Plus Minus'  # Replace with your default x-axis column
    ),
    dcc.Dropdown(
        id='y-column',
        options=[{'label': col, 'value': col} for col in overall.columns],
        value='Game Score'  # Replace with your default y-axis column
    ),
    # Dropdown for selecting color column
    dcc.Dropdown(
        id='color-column',
        options=[{'label': col, 'value': col} for col in overall.columns],  # All columns as options
        value="VORP"  # Default value or choose a specific column as the default
    ),
    dcc.Dropdown(
        id='size-column',
        options=[{'label': col, 'value': col} for col in numeric_cols],
        value="EFG%",  # Default value or choose a specific column as the default
        placeholder="Select column for point size"
    ),
    # Dropdown for multiple Name selection
    dcc.Dropdown(
        id='name-dropdown',
        options=name_options,
        value=[],  # Default value, can be an empty list or a list of default names
        multi=True,  # Enable multi-selection
        placeholder="Select Names"  # A placeholder text when no option is selected
    ),
    # Dropdown for multiple Position selection
    dcc.Dropdown(
        id='position-dropdown',
        options=position_options,
        value=[],  # Default value
        multi=True,  # Enable multi-selection
        placeholder="Select Positions"
    ),
    dcc.Dropdown(
    id='team-dropdown',
    options=team_options,
    value=[],  # Default value, can be an empty list or a list of default teams
    multi=True,  # Enable multi-selection
    placeholder="Select Teams"  # A placeholder text when no option is selected
    ),
    # Add a toggle switch for turning on/off text labels on the scatter plot
    dcc.Checklist(
        id='toggle-text-labels',  # ID to use in the callback
        options=[
            {'label': 'Toggle Text Labels', 'value': 'on'}  # The value 'on' represents the toggle being active
        ],
        value=[],  # Default is off, so empty list
        labelStyle={'display': 'block'}  # Style it so it appears as a block element
    ),
    dcc.Checklist(
    id='include-drafted-toggle',  # ID to use in the callback
    options=[
        {'label': 'Include Drafted', 'value': 'include'}  # The value 'include' represents the toggle being active
    ],
    value=[],  # Default is off, so empty list
    labelStyle={'display': 'block'}  # Style it so it appears as a block element
    ),
    # Scatter plot with trendline
    dcc.Graph(id='scatter-plot'),
    # Filters for numeric columns only
] + [html.Div([
        html.Label(col),
        dcc.RangeSlider(
            id='filter-slider-' + col,
            min=overall[col].min(),
            max=overall[col].max(),
            value=[5, overall[col].max()] if col == 'Minutes' else [overall[col].min(), overall[col].max()],
            step=(overall[col].max() - overall[col].min()) / 100,  # Adjust step relative to range
            marks=create_slider_marks(overall[col].min(), overall[col].max())  # Use the new function
        )
    ], id='filter-div-' + col) for col in overall.columns if pd.api.types.is_numeric_dtype(overall[col])]
)

# Callback to update the scatter plot
# Adjust your callback to use numeric columns only
@app.callback(
    Output('scatter-plot', 'figure'),
    [
        Input('x-column', 'value'),
        Input('y-column', 'value'),
        Input('color-column', 'value'),
        Input('size-column', 'value'),
        Input('name-dropdown', 'value'),
        Input('position-dropdown', 'value'),
        Input('team-dropdown', 'value'),  # Adding the team dropdown as an input
        Input('toggle-text-labels', 'value'),
        Input('include-drafted-toggle', 'value')
    ] + [Input('filter-slider-' + col, 'value') for col in numeric_cols]
)
def update_scatter_plot(x_col, y_col, color_col, size_col, selected_names, selected_positions, selected_teams, toggle_text_labels, include_drafted, *filter_values):
    try:
        filtered_df = overall.copy()

        if size_col and size_col in filtered_df.columns:
            # Preprocess size column data
            filtered_df['size'] = preprocess_size_data(filtered_df[size_col])
            size_col = 'size'  # Update the size_col to the new preprocessed 'size' column
        else:
            size_col = None  # No size adjustment if the column isn't valid

        # Handle 'Select All' cases
        if 'all' in selected_names:
            selected_names = overall['Name'].unique()

        if 'all' in selected_positions:
            selected_positions = overall['Position'].unique()

        # Filter by selected names and positions
        if selected_names:
            filtered_df = filtered_df[filtered_df['Name'].isin(selected_names)]

        if selected_positions:
            filtered_df = filtered_df[filtered_df['Position'].isin(selected_positions)]

        # Apply filters to columns based on the range sliders
        for col, value in zip(numeric_cols, filter_values):
            filtered_df = filtered_df[(filtered_df[col] >= value[0]) & (filtered_df[col] <= value[1])]

        if filtered_df.empty:
            return px.scatter(title="No data to display with current filter settings")

        # Check the state of 'include-drafted-toggle'
        if 'include' in include_drafted:
            # If 'include' is in the toggle value, include all players regardless of 'Draft Status'
            # In other words, do nothing special to filtered_df, as it already contains all players
            pass  # Or explicitly: filtered_df = overall
        else:
            # If 'include' is NOT in the toggle value (i.e., it's unchecked),
            # include all players UNLESS 'Draft Status' for that player is 'Drafted'
            filtered_df = filtered_df[filtered_df['Draft Status'] != 'Drafted']

        # if 'include' not in include_drafted:
        #     filtered_df = filtered_df[filtered_df['Draft Status'] == 'Available']

        # # Ensure that drafted individuals are included by default and only excluded when toggle is set
        # if 'include' in include_drafted:
        #     # if 'include' is selected, do nothing special, keep all individuals (both drafted and available)
        #     pass
        # else:
        #     # When 'include' is not in the toggle value, filter out the drafted individuals
        #     filtered_df = filtered_df[filtered_df['Draft Status'] != 'Drafted']


        # Filter by selected teams
        if selected_teams:
            filtered_df = filtered_df[filtered_df['Team'].isin(selected_teams)]

        # Determine if text labels should be added based on the checkbox
        text_labels = filtered_df['Name'] if 'on' in toggle_text_labels else None  # Adjust this line to use the toggle

        # Create and return the scatter plot
        fig = px.scatter(
            filtered_df,
            x=x_col,
            y=y_col,
            color=color_col if color_col in filtered_df.columns else None,
            size=size_col,  # This now uses the preprocessed 'size' column
            text=text_labels,
            hover_data=['Name', 'Position', 'Draft Status', 'Rating', 'Points'],
            trendline='ols',  # This adds a single trendline for all points
            # Optional: Customize trendline color or style if needed
        )

        # Customize symbols based on 'Draft Status' without affecting the trendline
        for trace in fig.data:
            if trace.name == 'Drafted':
                trace.marker.symbol = 'square'
            else:
                trace.marker.symbol = 'circle'

        # Adjust text font to be small and less obtrusive
        fig.update_traces(textposition='top center', textfont_size=8)

        return fig
    except Exception as e:
        print(f"Error in update_scatter_plot: {e}")  # Log the error
        raise e  # Reraise the exception to see the stack trace in your console
        return px.scatter(title="Error generating the plot")  # Return an empty plot with an error message


# Run the app
if __name__ == "__main__":
    app.run_server(debug=False, jupyter_mode='external', host='localhost', port=1050)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
overall

,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM,Team,Round Picked
0,Kareem Abdul-Jabbar,29.68,4.92,16.12,1.76,1.68,4.08,5.76,1.44,1.22,...,2.80,39.52,36.64,28.48,99,C,Drafted,0.751012,Blake B,1
1,Shareef Abdur-Rahim,2.00,1.00,4.00,1.00,0.00,0.00,0.00,1.00,1.00,...,0.00,17.00,8.27,2.90,86,PF,Drafted,0.117647,Hayden R,12
2,Steven Adams,1.29,0.79,2.54,0.38,0.46,0.50,0.96,0.08,4.50,...,0.13,10.50,12.69,2.55,82,C,Drafted,0.122857,Elise D,10
3,Bam Adebayo,7.72,0.88,4.84,1.00,0.92,0.36,1.28,0.32,3.13,...,1.68,17.60,23.37,8.22,87,PF,Drafted,0.438636,Brian R,7
4,Mark Aguirre,3.44,0.08,1.48,0.80,0.12,0.08,0.20,0.36,2.22,...,0.12,13.52,8.01,2.20,88,SF,Available,0.254438,None,Unpicked
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Zion Williamson,11.28,1.08,3.56,2.72,0.20,0.16,0.36,1.16,2.34,...,2.24,29.92,15.08,9.32,92,PF,Drafted,0.377005,Brian R,3
322,Kevin Willis,2.67,0.67,3.67,1.00,0.00,0.50,0.50,0.33,3.00,...,0.00,13.83,12.82,3.55,87,PF,Available,0.193059,None,Unpicked
323,James Worthy,10.24,1.24,6.80,2.40,0.80,0.36,1.16,1.08,2.22,...,1.76,35.48,14.16,10.19,95,SF,Drafted,0.288613,Garrett H,4
324,Ming Yao,13.24,4.00,14.04,2.16,1.24,3.00,4.24,0.84,2.57,...,0.88,34.20,28.72,19.00,91,C,Drafted,0.387135,Jorge L,3


In [ ]:
print(overall['Draft Status'].unique())

['Drafted' 'Available']


In [ ]:
overall[['Name', 'Points', 'Draft Status', 'Team']]

,Name,Points,Draft Status,Team
0,Kareem Abdul-Jabbar,29.68,Drafted,Blake B
1,Shareef Abdur-Rahim,2.00,Drafted,Hayden R
2,Steven Adams,1.29,Drafted,Elise D
3,Bam Adebayo,7.72,Drafted,Brian R
4,Mark Aguirre,3.44,Available,None
...,...,...,...,...
321,Zion Williamson,11.28,Drafted,Brian R
322,Kevin Willis,2.67,Available,None
323,James Worthy,10.24,Drafted,Garrett H
324,Ming Yao,13.24,Drafted,Jorge L


In [ ]:
overall['Draft Status'].fillna('Available')

0        Drafted
1        Drafted
2        Drafted
3        Drafted
4      Available
         ...    
321      Drafted
322    Available
323      Drafted
324      Drafted
325      Drafted
Name: Draft Status, Length: 326, dtype: object

In [ ]:
len(overall.columns)

35

In [ ]:
len(merged_df.columns)

35

In [ ]:
overall[overall['Name']== 'Jason Kidd']

,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM,Team,Round Picked
169,Jason Kidd,12.84,0.16,2.32,8.12,1.68,0.12,1.8,1.8,4.51,...,0.0,36.2,15.55,11.84,95,PG,Drafted,0.354696,Adrian R,4


In [ ]:
team_averages = merged_df.groupby('Team').mean()
team_averages['Team'] = team_averages.index
team_averages

<ipython-input-19-89bcdf33d289>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,Turnovers Forced,...,Fouls,Plus Minus,VORP,Dunks,Minutes,PER,Game Score,Rating,PPM,Team
Team,,,,,,,,,,,,,,,,,,,,,
Adrian R,9.285000,0.854167,4.221667,2.975833,0.660000,0.491667,1.152500,0.794167,4.295000,0.358333,...,1.217500,-3.372500,-1.181667,0.541667,24.114167,15.445833,8.344167,91.666667,0.379743,Adrian R
Andrew N,11.495455,1.450909,6.595455,2.920000,0.939091,0.915455,1.854545,0.985455,3.064000,0.869091,...,1.443636,-0.368182,0.588182,0.965455,29.204545,19.397273,11.741818,92.545455,0.395375,Andrew N
Arijit B,10.050833,0.887500,5.015000,2.170000,0.746667,0.823333,1.570833,0.577500,3.553333,0.994167,...,1.105833,2.115000,2.447500,0.965000,23.565000,17.813333,9.468333,89.000000,0.373933,Arijit B
Blake B,12.676923,0.705385,3.986923,3.712308,0.675385,0.733077,1.408462,1.141538,4.009231,0.265385,...,1.287692,2.821538,2.540000,0.615385,24.645385,20.286154,10.794615,89.923077,0.471382,Blake B
Brett N,15.310000,0.215455,2.073636,3.354545,0.310000,0.047273,0.357273,0.939091,3.484545,-0.580909,...,0.932727,-0.732727,0.856364,0.453636,26.642727,17.669091,10.247273,88.272727,0.567637,Brett N
Brian R,10.844167,1.256667,5.604167,2.473333,0.710000,0.896667,1.605833,0.643333,4.027500,0.963333,...,1.474167,-2.504167,-0.401667,1.104167,26.158333,18.442500,10.710833,91.500000,0.385649,Brian R
David B,11.350833,1.413333,6.501667,2.024167,0.688333,1.235833,1.923333,0.655000,3.360833,1.268333,...,1.212500,1.694167,2.325833,0.736667,24.441667,22.134167,11.292500,90.583333,0.435688,David B
Elise D,12.754444,0.578889,3.713333,2.990000,0.480000,0.495556,0.976667,1.057778,3.298889,-0.081111,...,1.017778,4.471111,3.810000,0.296667,23.223333,20.606667,10.674444,89.666667,0.478141,Elise D
Esben P,10.417500,0.758333,4.435000,2.746667,0.788333,0.672500,1.459167,0.716667,3.966667,0.741667,...,1.205833,5.502500,5.141667,0.676667,25.735833,17.317500,9.685000,92.000000,0.371244,Esben P


In [ ]:
# Filtering out only drafted players and converting "Round Picked" to a numeric value for sorting and grouping
# team_averages['Round Picked'] = pd.to_numeric(merged_df['Round Picked'], errors='coerce')
sorted_round = merged_df[merged_df['Draft Status'] == 'Drafted'].sort_values(by=['Team', 'Round Picked'])
sorted_round

# Group by 'Team' and take the mean of the first 5 drafted players for each team
# team_averages = team_averages.groupby('Team').head(5).groupby('Team').mean()
# team_averages

,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM,Team,Round Picked
301,Dwyane Wade,24.68,0.24,2.20,6.20,0.80,0.24,1.04,2.08,2.98,...,2.52,35.92,24.19,18.21,97,SG,Drafted,0.687082,Adrian R,1
99,Kevin Garnett,18.42,4.17,15.46,2.63,1.04,1.58,2.63,1.13,2.33,...,1.46,35.21,26.27,18.73,98,PF,Drafted,0.523147,Adrian R,2
100,Marc Gasol,7.84,2.04,10.72,2.20,1.08,1.76,2.84,0.40,5.50,...,0.24,37.56,14.33,10.68,93,C,Drafted,0.208733,Adrian R,3
169,Jason Kidd,12.84,0.16,2.32,8.12,1.68,0.12,1.80,1.80,4.51,...,0.00,36.20,15.55,11.84,95,PG,Drafted,0.354696,Adrian R,4
202,George Mikan,8.88,2.48,11.40,1.36,0.72,1.48,2.20,0.76,1.79,...,1.44,33.68,17.41,11.34,94,C,Drafted,0.263658,Adrian R,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,Arvydas Sabonis,3.92,0.64,3.84,0.48,0.36,0.68,1.04,0.12,4.00,...,0.16,12.76,20.43,5.16,88,C,Drafted,0.307210,William K,7
54,Tyson Chandler,3.16,1.32,3.96,0.40,0.68,0.56,1.24,0.12,3.33,...,0.28,13.00,19.71,4.95,85,C,Drafted,0.243077,William K,8
17,Lamelo Ball,5.31,0.19,1.63,3.50,0.38,0.13,0.50,1.00,3.50,...,0.19,16.38,12.12,4.25,87,PG,Drafted,0.324176,William K,9
287,Jason Terry,6.28,0.00,0.39,1.83,0.11,0.00,0.11,0.33,5.50,...,0.00,13.06,18.10,4.92,84,SG,Drafted,0.480858,William K,10


In [ ]:
tmp = sorted_round.groupby('Team').head(5)

In [ ]:
tmp[tmp['Team']=='Adrian R']

,Name,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,...,Dunks,Minutes,PER,Game Score,Rating,Position,Draft Status,PPM,Team,Round Picked
301,Dwyane Wade,24.68,0.24,2.20,6.20,0.80,0.24,1.04,2.08,2.98,...,2.52,35.92,24.19,18.21,97,SG,Drafted,0.687082,Adrian R,1
99,Kevin Garnett,18.42,4.17,15.46,2.63,1.04,1.58,2.63,1.13,2.33,...,1.46,35.21,26.27,18.73,98,PF,Drafted,0.523147,Adrian R,2
100,Marc Gasol,7.84,2.04,10.72,2.20,1.08,1.76,2.84,0.40,5.50,...,0.24,37.56,14.33,10.68,93,C,Drafted,0.208733,Adrian R,3
169,Jason Kidd,12.84,0.16,2.32,8.12,1.68,0.12,1.80,1.80,4.51,...,0.00,36.20,15.55,11.84,95,PG,Drafted,0.354696,Adrian R,4
202,George Mikan,8.88,2.48,11.40,1.36,0.72,1.48,2.20,0.76,1.79,...,1.44,33.68,17.41,11.34,94,C,Drafted,0.263658,Adrian R,5


In [ ]:
team_averages

,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,Turnovers Forced,...,Fouls,Plus Minus,VORP,Dunks,Minutes,PER,Game Score,Rating,PPM,Team
Team,,,,,,,,,,,,,,,,,,,,,
Adrian R,9.285000,0.854167,4.221667,2.975833,0.660000,0.491667,1.152500,0.794167,4.295000,0.358333,...,1.217500,-3.372500,-1.181667,0.541667,24.114167,15.445833,8.344167,91.666667,0.379743,Adrian R
Andrew N,11.495455,1.450909,6.595455,2.920000,0.939091,0.915455,1.854545,0.985455,3.064000,0.869091,...,1.443636,-0.368182,0.588182,0.965455,29.204545,19.397273,11.741818,92.545455,0.395375,Andrew N
Arijit B,10.050833,0.887500,5.015000,2.170000,0.746667,0.823333,1.570833,0.577500,3.553333,0.994167,...,1.105833,2.115000,2.447500,0.965000,23.565000,17.813333,9.468333,89.000000,0.373933,Arijit B
Blake B,12.676923,0.705385,3.986923,3.712308,0.675385,0.733077,1.408462,1.141538,4.009231,0.265385,...,1.287692,2.821538,2.540000,0.615385,24.645385,20.286154,10.794615,89.923077,0.471382,Blake B
Brett N,15.310000,0.215455,2.073636,3.354545,0.310000,0.047273,0.357273,0.939091,3.484545,-0.580909,...,0.932727,-0.732727,0.856364,0.453636,26.642727,17.669091,10.247273,88.272727,0.567637,Brett N
Brian R,10.844167,1.256667,5.604167,2.473333,0.710000,0.896667,1.605833,0.643333,4.027500,0.963333,...,1.474167,-2.504167,-0.401667,1.104167,26.158333,18.442500,10.710833,91.500000,0.385649,Brian R
David B,11.350833,1.413333,6.501667,2.024167,0.688333,1.235833,1.923333,0.655000,3.360833,1.268333,...,1.212500,1.694167,2.325833,0.736667,24.441667,22.134167,11.292500,90.583333,0.435688,David B
Elise D,12.754444,0.578889,3.713333,2.990000,0.480000,0.495556,0.976667,1.057778,3.298889,-0.081111,...,1.017778,4.471111,3.810000,0.296667,23.223333,20.606667,10.674444,89.666667,0.478141,Elise D
Esben P,10.417500,0.758333,4.435000,2.746667,0.788333,0.672500,1.459167,0.716667,3.966667,0.741667,...,1.205833,5.502500,5.141667,0.676667,25.735833,17.317500,9.685000,92.000000,0.371244,Esben P


In [ ]:
# prompt: Using dataframe team_averages: create a plotly scatterplot with 'Plus Minus' on x axis, 'Vorp' on y axis,  colored by 'Team'

import plotly.express as px
fig = px.scatter(team_averages, x="Points", y="Rebounds", color="Team", size='Points')
fig.show()


In [ ]:
fig = px.scatter(team_averages, x="PER", y="Plus Minus", color="Team", size='Points')
fig.show()

In [ ]:
fig = px.scatter(team_averages, x="3pt Attempted", y="FG Attempted", color="Team", size='Points')
fig.show()

In [ ]:
fig = px.scatter_3d(team_averages, x='Plus Minus', y='Points', z='PER', color='Team')
fig.show()

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Given data for the teams

# Creating DataFrame
df = team_averages

# Extracting features for clustering
X = df.drop("Team", axis=1)

# Using KMeans clustering with an assumed 3 clusters for illustration
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X)

# Assigning the cluster labels to the dataframe
df['Cluster'] = kmeans.labels_

df[['Team', 'Cluster']]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



,Team,Cluster
Team,,
Adrian R,Adrian R,0
Andrew N,Andrew N,1
Arijit B,Arijit B,4
Blake B,Blake B,4
Brett N,Brett N,3
Brian R,Brian R,0
David B,David B,1
Elise D,Elise D,4
Esben P,Esben P,4


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans


# Creating DataFrame
df = team_averages

# Extracting features for clustering
X = df.drop("Team", axis=1)

# Using KMeans clustering with an assumed 3 clusters for illustration
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(X)

# Assigning the cluster labels to the dataframe
df['Cluster'] = kmeans.labels_

df[['Team', 'Cluster']]

# Reduce dimensions to 2D for visualization using PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data=principalComponents, columns=['PC1', 'PC2'])

# Resetting index to ensure alignment
principalDf = principalDf.reset_index(drop=True)
df_reset = df.reset_index(drop=True)

# Add the cluster and team information back to the PCA dataframe
principalDf['Cluster'] = kmeans.labels_
principalDf['Team'] = df_reset['Team']  # Ensure this is aligned by position

# Plotting
plt.figure(figsize=(12,8))
sns.scatterplot(x="PC1", y="PC2", hue="Cluster", data=principalDf, palette="viridis", s=100).set_title('Team Cluster Distribution')
for line in range(0,principalDf.shape[0]):
     plt.text(principalDf['PC1'][line]+0.2, principalDf['PC2'][line], principalDf['Team'][line], horizontalalignment='left', size='medium', color='black', weight='semibold')
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()


In [ ]:
df['Team']

Team
Adrian R      Adrian R
Andrew N      Andrew N
Arijit B      Arijit B
Blake B        Blake B
Brett N        Brett N
Brian R        Brian R
David B        David B
Elise D        Elise D
Esben P        Esben P
Garrett H    Garrett H
Garrett S    Garrett S
Hayden R      Hayden R
Jake E          Jake E
Jordan B      Jordan B
Jorge L        Jorge L
Lars L          Lars L
Michael G    Michael G
Phil N          Phil N
Quinn L        Quinn L
Steven B      Steven B
Wei Ren G    Wei Ren G
William K    William K
Name: Team, dtype: object

In [ ]:
principalDf

,PC1,PC2,Cluster,Team
0,0.582812,8.534464,2,NaN
1,-2.660011,0.890174,0,NaN
2,0.101072,-4.254036,0,NaN
3,3.961822,-5.615291,1,NaN
4,6.246471,-4.982974,1,NaN
5,-2.336331,6.325189,2,NaN
6,-2.160127,-0.653726,0,NaN
7,-3.539114,-5.409253,0,NaN
8,-4.058000,-7.245181,0,NaN
9,7.882018,0.682936,1,NaN


In [ ]:
principalDf['Team']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
Name: Team, dtype: object

In [ ]:
principalDf

,PC1,PC2,Cluster,Team
0,0.582812,8.534464,2,NaN
1,-2.660011,0.890174,0,NaN
2,0.101072,-4.254036,0,NaN
3,3.961822,-5.615291,1,NaN
4,6.246471,-4.982974,1,NaN
5,-2.336331,6.325189,2,NaN
6,-2.160127,-0.653726,0,NaN
7,-3.539114,-5.409253,0,NaN
8,-4.058000,-7.245181,0,NaN
9,7.882018,0.682936,1,NaN


In [ ]:
# Calculate feature importances from the kmeans clustering
# Since KMeans doesn't provide straightforward feature importances like some supervised algorithms,
# we can use the explained variance ratio from PCA as a proxy to understand which features
# have the most influence on the variation in the dataset.

# Calculating the percentage of variance explained by each of the selected components
explained_variance = pca.explained_variance_ratio_

# Create a bar plot for the explained variance by each principal component
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance*100, color='green')
plt.ylabel('Percentage of explained variance')
plt.xlabel('Principal Component')
plt.title('PCA - Percentage of Explained Variance by Each Component')
plt.xticks([1, 2])
plt.show()

# The actual feature importances are derived from the loadings of the original variables on the principal components
# Getting the absolute value of loadings for each feature on the principal components
loadings = pd.DataFrame(abs(pca.components_.T), columns=['PC1', 'PC2'], index=X.columns)

# Creating a bar plot for the feature importances for the first two principal components
plt.figure(figsize=(12, 8))
sns.barplot(data=loadings.sort_values('PC1', ascending=False), x='PC1', y=loadings.index).set_title('Feature Importances for PC1')
plt.show()

# Creating a bar plot for the feature importances for the first two principal components
plt.figure(figsize=(12, 8))
sns.barplot(data=loadings.sort_values('PC2', ascending=False), x='PC2', y=loadings.index).set_title('Feature Importances for PC2')
plt.show()


In [ ]:
# Replace a few troublesome typos
merged_df['Name'].replace('Ming Yao', 'Yao Ming', inplace=True)
merged_df['Name'].replace('Jaren Jr Jackson', 'Jaren Jackson Jr', inplace=True)

s22_draft = gc.open('Season 22 Draft')
dataframes = {}
for sheet in s22_draft.worksheets():
    dataframes[sheet.title] = pd.DataFrame(sheet.get_all_values())

tmp = dataframes['DRAFT!']

# clean up the df and use the titles in the first row ('FT Made', 'Dunks' etc) the column names
tmp = tmp.rename(columns=tmp.iloc[0])
tmp = tmp.drop(tmp.index[0])

# Melting dataframe to long format
melted_df = tmp.melt(id_vars="Round/Team", var_name="Team", value_name="Name")

# Splitting the team name from the player's name in parentheses
melted_df["Team"] = melted_df["Team"].str.extract(r'(.*) \(')[0]

# Dropping empty rows based on 'Name' column
melted_df = melted_df[melted_df["Name"] != ""]

melted_df

,Round/Team,Team,Name
0,1,Jordan B,Giannis Antetokounmpo
1,2,Jordan B,Bill Walton
2,3,Jordan B,Artis Gilmore
3,4,Jordan B,Michael Redd
4,5,Jordan B,Andrei Kirilenko
...,...,...,...
255,4,Wei Ren G,Rudy Gobert
256,5,Wei Ren G,Mel Daniels
257,6,Wei Ren G,James Donaldson
258,7,Wei Ren G,Dave Debusschere


In [ ]:
overall.corr().loc['Rating']

<ipython-input-19-446811715c4a>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Points              0.707078
Offensive Reb       0.393126
Rebounds            0.452068
Assists             0.593494
Steals              0.605205
Blocks              0.298510
Stocks              0.485815
Turnovers           0.556947
A/TO                0.031898
Turnovers Forced    0.171404
FG Made             0.706822
FG Attempted        0.702240
FG%                 0.001968
3pt Made            0.358733
3pt Attempted       0.353902
3FG%                0.122957
FT Made             0.702251
FT Attempted        0.693867
FT%                 0.335526
TS%                -0.015880
EFG%               -0.351174
Fouls               0.615016
Plus Minus          0.081318
VORP                0.176082
Dunks               0.475673
Minutes             0.726059
PER                 0.439485
Game Score          0.717303
Rating              1.000000
PPM                 0.424313
Name: Rating, dtype: float64

In [ ]:
overall.corr()

<ipython-input-98-ffe5dc3f8d32>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Stocks,Turnovers,A/TO,Turnovers Forced,...,EFG%,Fouls,Plus Minus,VORP,Dunks,Minutes,PER,Game Score,Rating,PPM
Points,1.000000,0.339143,0.428214,0.642012,0.568600,0.356228,0.509917,0.672351,-0.092272,0.130881,...,-0.235231,0.636095,0.205651,0.314791,0.569785,0.799123,0.624782,0.928033,0.707078,0.757848
Offensive Reb,0.339143,1.000000,0.952911,0.011244,0.510013,0.823462,0.813052,0.039376,-0.094821,0.784859,...,-0.070763,0.693659,0.107559,0.158304,0.633554,0.526712,0.460531,0.609151,0.393126,0.074582
Rebounds,0.428214,0.952911,1.000000,0.102971,0.548480,0.859672,0.856065,0.131230,-0.093879,0.776314,...,-0.091478,0.754712,0.165497,0.227781,0.690955,0.631745,0.514036,0.694059,0.452068,0.120879
Assists,0.642012,0.011244,0.102971,1.000000,0.451346,0.043498,0.235941,0.847059,0.123110,-0.238427,...,-0.321324,0.452577,0.173577,0.290146,0.277054,0.651901,0.284304,0.600812,0.593494,0.364131
Steals,0.568600,0.510013,0.548480,0.451346,1.000000,0.460141,0.779496,0.420866,-0.023756,0.538871,...,-0.235308,0.624472,0.082185,0.153362,0.501171,0.701665,0.436557,0.678780,0.605205,0.267150
Blocks,0.356228,0.823462,0.859672,0.043498,0.460141,1.000000,0.914828,0.107541,-0.142898,0.847761,...,-0.011568,0.685341,0.285334,0.329007,0.617176,0.495456,0.478099,0.615449,0.298510,0.108000
Stocks,0.509917,0.813052,0.856065,0.235941,0.779496,0.914828,1.000000,0.267249,-0.113048,0.843264,...,-0.114991,0.767496,0.238553,0.301733,0.663408,0.668646,0.535891,0.742941,0.485815,0.197714
Turnovers,0.672351,0.039376,0.131230,0.847059,0.420866,0.107541,0.267249,1.000000,-0.278039,-0.292583,...,-0.347148,0.449271,0.121746,0.249169,0.274066,0.632522,0.200326,0.587414,0.556947,0.381310
A/TO,-0.092272,-0.094821,-0.093879,0.123110,-0.023756,-0.142898,-0.113048,-0.278039,1.000000,0.038545,...,0.107570,-0.082976,0.039988,0.013646,-0.082756,-0.059665,0.103162,-0.046678,0.031898,-0.052989
Turnovers Forced,0.130881,0.784859,0.776314,-0.238427,0.538871,0.847761,0.843264,-0.292583,0.038545,1.000000,...,0.079329,0.511021,0.168727,0.160385,0.505392,0.310761,0.419955,0.409521,0.171404,-0.016677


In [ ]:
%%time

import pandas as pd
from fuzzywuzzy import process, fuzz
import numpy as np

s22_draft = gc.open('Season 21 Box Scores')

player_stats = {}  # A dictionary to hold aggregated stats
unique_player_names = set()  # Set to hold unique player names

# Define the statistical categories you want to aggregate
stats_columns = [
    'Points', 'Offensive Reb', 'Rebounds', 'Assists', 'Steals',
    'Blocks', 'Turnovers', 'FG Made', 'FG Attempted', '3pt Made',
    '3pt Attempted', 'FT Made', 'FT Attempted', 'Fouls',
    'Plus Minus', 'Dunks', 'Minutes', 'Game Score'
]

def get_fuzzy_match(player_name, choices, scorer=fuzz.token_sort_ratio, cutoff=85):
    if player_name in choices:  # If the exact name is already in the set, just use it
        return player_name

    match = process.extractOne(player_name, choices, scorer=scorer)  # This returns a tuple (match, score) or None
    if match and match[1] > cutoff:  # Ensure match is not None and score is above cutoff
        return match[0]  # Return the matching string

    return player_name.title()  # Default return if no match or below cutoff


for sheet in s22_draft.worksheets():
    df = pd.DataFrame(sheet.get_all_values())
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df.reset_index(drop=True, inplace=True)

    if df.empty:
        print(f"No data in sheet {sheet.title}. Skipping.")
        continue

    separating_index = 11  # given the first team is in the first 10 rows after the header

    team_1 = df.iloc[:separating_index]
    team_2 = df.iloc[separating_index:]

    for team in [team_1, team_2]:
        for _, row in team.iterrows():
            original_name = row['Name']

            # Skip empty or malformed rows
            if pd.isna(original_name) or original_name.strip() == '':
                continue

            clean_name = original_name.title().strip()
            player_name = get_fuzzy_match(clean_name, unique_player_names)
            unique_player_names.add(player_name)  # Always add the possibly corrected name

            row_numeric = row[stats_columns].apply(pd.to_numeric, errors='coerce')
            if player_name in player_stats:
                player_stats[player_name]['Games'] += 1
                for stat in stats_columns:
                    value = row_numeric[stat]
                    player_stats[player_name][stat] += value if not np.isnan(value) else 0
            else:
                player_stats[player_name] = {'Games': 1}
                for stat in stats_columns:
                    value = row_numeric[stat]
                    player_stats[player_name][stat] = value if not np.isnan(value) else 0

for player, stats in player_stats.items():
    for stat in stats_columns:
        if player_stats[player]['Games'] > 0:
            player_stats[player][stat] /= player_stats[player]['Games']
        else:
            player_stats[player][stat] = 0

aggregated_df = pd.DataFrame.from_dict(player_stats, orient='index')
aggregated_df

No data in sheet W21 JGS @ BW. Skipping.
                          Games     Points  Offensive Reb  Rebounds   Assists  \
Harden, James                24  36.916667       0.708333     6.125  7.583333   
Gilgeous-Alexander, Shai     25  22.840000       0.680000     3.280  5.480000   
Donaldson, James             25   8.480000       2.320000     8.880  1.080000   
Redd, Michael                25  12.800000       0.200000     1.880  1.920000   
Brunson, Jalen               25   6.920000       0.200000     0.440  1.920000   
...                         ...        ...            ...       ...       ...   
Coleman, Derrick              1   2.000000       0.000000     2.000  0.000000   
Divac, Clade                  1   4.000000       1.000000     4.000  0.000000   
Martin, Kevin                 4   2.500000       0.000000     0.250  1.500000   
Rivers, Doc                   1   2.000000       0.000000     0.000  1.000000   
Kennard, Luke                 1   7.000000       0.000000     0.000 

In [ ]:
aggregated_df.columns

Index(['Games', 'Points', 'Offensive Reb', 'Rebounds', 'Assists', 'Steals',
       'Blocks', 'Turnovers', 'FG Made', 'FG Attempted', '3pt Made',
       '3pt Attempted', 'FT Made', 'FT Attempted', 'Fouls', 'Plus Minus',
       'Dunks', 'Minutes', 'Game Score'],
      dtype='object')

In [ ]:
# https://www.reddit.com/r/nba/comments/arqlqb/a_guide_to_advanced_stats/
# https://www.bball-index.com/player-impact-plus-minus/
# https://github.com/rd11490/NBA_Tutorials/tree/master

aggregated_df['Stocks'] = aggregated_df['Steals'] + aggregated_df['Blocks']

aggregated_df['A/TO'] = aggregated_df['Assists']/aggregated_df['Turnovers']

aggregated_df['FG%'] = aggregated_df['FG Made']/aggregated_df['FG Attempted']

aggregated_df['3pt%'] = aggregated_df['3pt Made']/aggregated_df['3pt Attempted']

aggregated_df['FT%'] = aggregated_df['FT Made']/aggregated_df['FT Attempted']

aggregated_df['TS%'] = (aggregated_df['Points'])/(2*(aggregated_df['FG Attempted']+(0.44*aggregated_df['FT Attempted'])))

aggregated_df['EFG%'] = ((aggregated_df['Points']-aggregated_df['FT Made'])/2)/aggregated_df['FT Attempted']

aggregated_df['Plus Minus'] = ((aggregated_df['Points']-aggregated_df['FT Made'])/2)/aggregated_df['FT Attempted']

aggregated_df['Vorp'] = (aggregated_df['Plus Minus']+2)*(aggregated_df['Minutes']/48)

aggregated_df['PER'] = (((aggregated_df['FG Made']*85.91)+(aggregated_df['Steals']*53.897)+(aggregated_df['3pt Made']*51.757)+(aggregated_df['FT Made']*46.845)+(aggregated_df['Blocks']*39.19)+(aggregated_df['Offensive Reb']*39.19)+(aggregated_df['Assists']*34.677)+((aggregated_df['Rebounds']-aggregated_df['Offensive Reb'])*14.707)-(aggregated_df['Fouls']*17.174)-((aggregated_df['FT Attempted']-aggregated_df['FT Made'])*20.091)-((aggregated_df['FG Attempted']-aggregated_df['FG Made'])*39.19)-(aggregated_df['Turnovers']*53.897))*(1/aggregated_df['Minutes']))

aggregated_df['PPM'] = aggregated_df['Points']/aggregated_df['Minutes']


In [ ]:
 aggregated_df

,Games,Points,Offensive Reb,Rebounds,Assists,Steals,Blocks,Turnovers,FG Made,FG Attempted,...,Stocks,A/TO,FG%,3pt%,FT%,TS%,EFG%,Vorp,PER,PPM
"Harden, James",24,36.916667,0.708333,6.125,7.583333,0.708333,0.083333,1.666667,12.291667,27.916667,...,0.791667,4.550000,0.440299,0.350785,0.835052,0.586475,1.865979,3.194802,33.161512,0.930672
"Gilgeous-Alexander, Shai",25,22.840000,0.680000,3.280,5.480000,0.640000,0.680000,1.600000,8.440000,16.280000,...,1.320000,3.425000,0.518428,0.531250,0.823529,0.621517,1.987395,2.864279,25.845248,0.662413
"Donaldson, James",25,8.480000,2.320000,8.880,1.080000,1.240000,2.080000,0.640000,3.600000,5.120000,...,3.320000,1.687500,0.703125,NaN,0.918919,0.734683,2.405405,2.389932,23.895175,0.325653
"Redd, Michael",25,12.800000,0.200000,1.880,1.920000,0.320000,0.080000,0.400000,4.400000,9.240000,...,0.400000,4.800000,0.476190,0.475410,0.866667,0.673401,10.233333,6.269583,19.185607,0.520325
"Brunson, Jalen",25,6.920000,0.200000,0.440,1.920000,0.120000,0.000000,0.520000,2.440000,4.840000,...,0.120000,3.692308,0.504132,0.571429,0.904762,0.664158,3.666667,1.898333,16.329575,0.430348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Coleman, Derrick",1,2.000000,0.000000,2.000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,NaN,1.000000,NaN,NaN,1.000000,inf,inf,49.075000,1.000000
"Divac, Clade",1,4.000000,1.000000,4.000,0.000000,0.000000,1.000000,0.000000,2.000000,2.000000,...,1.000000,NaN,1.000000,NaN,NaN,1.000000,inf,inf,34.643375,0.500000
"Martin, Kevin",4,2.500000,0.000000,0.250,1.500000,0.000000,0.000000,0.250000,0.750000,3.000000,...,0.000000,6.000000,0.250000,0.125000,1.000000,0.375375,1.166667,0.692708,6.337714,0.238095
"Rivers, Doc",1,2.000000,0.000000,0.000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,inf,1.000000,NaN,NaN,1.000000,inf,inf,34.471000,0.666667


In [ ]:
%%time

import pandas as pd
from fuzzywuzzy import process, fuzz
import numpy as np

s22_draft = gc.open('Season 21 Box Scores')

player_stats = {}  # A dictionary to hold aggregated stats
unique_player_names = set()  # Set to hold unique player names

# Define the statistical categories you want to aggregate
stats_columns = [
    'Points', 'Offensive Reb', 'Rebounds', 'Assists', 'Steals',
    'Blocks', 'Turnovers', 'FG Made', 'FG Attempted', '3pt Made',
    '3pt Attempted', 'FT Made', 'FT Attempted', 'Fouls',
    'Plus Minus', 'Dunks', 'Minutes', 'Game Score'
]

# Function to find the best match for a given player name
def get_fuzzy_match(player_name, choices, scorer=fuzz.token_sort_ratio, cutoff=85):
    match, score = process.extractOne(player_name.title(), choices, scorer=scorer)
    if score > cutoff:
        return match  # Only return a match if the score is above a certain cutoff
    return player_name.title()  # Return the original title-cased name if no good match

for sheet in s22_draft.worksheets():
    df = pd.DataFrame(sheet.get_all_values())

    # Clean up the DataFrame and use the titles in the first row as column names
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df.reset_index(drop=True, inplace=True)

    if df.empty:
        print(f"No data in sheet {sheet.title}. Skipping.")
        continue

    separating_index = 11  # given the first team is in the first 10 rows after the header

    team_1 = df.iloc[:separating_index]
    team_2 = df.iloc[separating_index:]

    for team in [team_1, team_2]:
        for _, row in team.iterrows():
            original_name = row['Name']

            # Skip empty or malformed rows
            if pd.isna(original_name) or original_name.strip() == '':
                continue

            unique_player_names.add(original_name.title())
            player_name = get_fuzzy_match(original_name, unique_player_names)
            row_numeric = row[stats_columns].apply(pd.to_numeric, errors='coerce')

            if player_name in player_stats:
                player_stats[player_name]['Games'] += 1
                for stat in stats_columns:
                    value = row_numeric[stat]
                    if not np.isnan(value):  # Check if the value is not NaN
                        player_stats[player_name][stat] += value
                    else:
                        player_stats[player_name][stat] += 0
            else:
                player_stats[player_name] = {'Games': 1}
                for stat in stats_columns:
                    value = row_numeric[stat]
                    player_stats[player_name][stat] = value if not np.isnan(value) else 0

    print(f"{sheet.title} Accumulated")

for player, stats in player_stats.items():
    for stat in stats_columns:
        if player_stats[player]['Games'] > 0:
            player_stats[player][stat] /= player_stats[player]['Games']
        else:
            player_stats[player][stat] = 0

aggregated_df = pd.DataFrame.from_dict(player_stats, orient='index')
print(aggregated_df)
# aggregated_df.to_excel('aggregated_player_stats.xlsx')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = merged_df.copy()
df = df.drop(columns=['Draft Status'])

# Drop rows with any NaN values
df_clean = df.dropna()
df = df_clean.copy()

# Assuming df is your DataFrame
X = df.drop(columns=['Name', 'Position', 'Rating'])  # Features
y = df['Rating']  # Target variable, assuming Rating is a composite score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model
model = RandomForestRegressor(n_estimators=1100, random_state=42, max_depth = 70)
model.fit(X_train, y_train)

# Predict the 'Rating' for each player
df['Predicted_Rating'] = model.predict(X)

# Rank players within each position based on predicted rating
df['Rank'] = df.groupby('Position')['Predicted_Rating'].rank(ascending=False)

# Now you can filter the dataframe for each position and see the top players
for position in df['Position'].unique():
    top_players = df[df['Position'] == position].sort_values(by='Rank').head(5)
    print(f"Top players for position {position}:")
    print(top_players[['Name', 'Predicted_Rating', 'Rank']])


Top players for position C:
                    Name  Predicted_Rating  Rank
0    Kareem Abdul-Jabbar         98.811818   1.0
51      Wilt Chamberlain         98.280000   2.0
150         Nikola Jokic         98.224545   3.0
209     Shaquille O'Neal         98.160909   4.0
175         Moses Malone         97.576364   5.0
Top players for position PF:
                      Name  Predicted_Rating  Rank
206          Dirk Nowitzki         98.687273   1.0
79              Tim Duncan         98.590909   2.0
7    Giannis Antetokounmpo         98.553636   3.0
267           Jayson Tatum         98.065455   4.0
22         Charles Barkley         97.470909   5.0
Top players for position SF:
               Name  Predicted_Rating  Rank
31       Larry Bird         98.533636   1.0
80     Kevin Durant         98.491818   2.0
164   Kawhi Leonard         98.430000   3.0
142    Lebron James         97.852727   4.0
221  Scottie Pippen         96.213636   5.0
Top players for position SG:
               Name  

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = merged_df.copy()

# Drop rows with any NaN values
df_clean = df.dropna()
df = df_clean.copy()

df = df.drop(columns=['Draft Status'])

# Assuming df is your DataFrame
X = df.drop(columns=['Name', 'Position', 'Rating'])  # Features
y = df['Rating']  # Target variable, assuming Rating is a composite score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model
model = RandomForestRegressor(n_estimators=1100, random_state=42, max_depth = 70)
model.fit(X_train, y_train)

# Predict the 'Rating' for each player
df['Predicted_Rating'] = model.predict(X)

# Rank players based on predicted rating across the entire dataset
df['Rank'] = df['Predicted_Rating'].rank(ascending=False)

# Now you can see the top players overall
top_players_overall = df.sort_values(by='Rank').head(50)
print("Top players overall:")
print(top_players_overall[['Name', 'Position', 'Predicted_Rating', 'Rating', 'Rank']])


Top players overall:
                        Name Position  Predicted_Rating  Rating  Rank
0        Kareem Abdul-Jabbar        C         98.811818    99.0   1.0
206            Dirk Nowitzki       PF         98.687273    99.0   2.0
79                Tim Duncan       PF         98.590909    99.0   3.0
7      Giannis Antetokounmpo       PF         98.553636    98.0   4.0
31                Larry Bird       SF         98.533636    99.0   5.0
80              Kevin Durant       SF         98.491818    99.0   6.0
164            Kawhi Leonard       SF         98.430000    99.0   7.0
51          Wilt Chamberlain        C         98.280000    99.0   8.0
150             Nikola Jokic        C         98.224545    99.0   9.0
209         Shaquille O'Neal        C         98.160909    99.0  10.0
119             James Harden       PG         98.130909    99.0  11.0
43               Kobe Bryant       SG         98.070000    99.0  12.0
140            Allen Iverson       PG         98.067273    99.0  13.0

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculating MAE, MSE, and R-squared
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)
print("R-squared:", r_squared)

MAE: 1.6960227272727277
MSE: 4.829709636953958
R-squared: 0.8018373270163333


In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
parameters = {
    'max_depth': [70, 80, 90, 100],
    'n_estimators': [900, 1000, 1100]
}
gridforest = GridSearchCV(model, parameters, cv = 3, n_jobs = -1, verbose = 1)
gridforest.fit(X_train, y_train)
gridforest.best_params_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


{'max_depth': 70, 'n_estimators': 900}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances from the model
importances = model.feature_importances_

# Sort the feature importances in descending order and get the indices
indices = np.argsort(importances)[::-1]

# Rearrange feature names so they match the sorted feature importances
names = [X.columns[i] for i in indices]

# Create a plot
plt.figure(figsize=(10, 6))
plt.title("Feature Importances")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), names, rotation=90)
plt.show()

# Print the feature importances
sorted_importances = [(names[i], importances[indices[i]]) for i in range(len(names))]
sorted_importances


In [ ]:
top_players_overall['Diff'] = top_players_overall['Predicted_Rating']-top_players_overall['Rating']

In [ ]:
sorted(top_players_overall['Name'])

['Allen Iverson',
 'Bill Russell',
 'Bill Walton',
 'Carmelo Anthony',
 'Charles Barkley',
 'Chris Paul',
 'Clyde Drexler',
 'Damian Lillard',
 'David Robinson',
 'Derrick Rose',
 'Devin Booker',
 'Dirk Nowitzki',
 'Dwight Howard',
 'Dwyane Wade',
 'Elgin Baylor',
 'Giannis Antetokounmpo',
 'James Harden',
 'Jayson Tatum',
 'Jerry West',
 'Jimmy Butler',
 'Joel Embiid',
 'John Havlicek',
 'John Stockton',
 'Julius Erving',
 'Kareem Abdul-Jabbar',
 'Karl Malone',
 'Kawhi Leonard',
 'Kevin Durant',
 'Kevin Garnett',
 'Kevin McHale',
 'Kobe Bryant',
 'Larry Bird',
 'Lebron James',
 'Luka Doncic',
 'Magic Johnson',
 'Michael Jordan',
 'Moses Malone',
 'Nikola Jokic',
 'Oscar Robertson',
 'Pete Maravich',
 'Reggie Miller',
 'Scottie Pippen',
 'Shai Gilgeous-Alexander',
 "Shaquille O'Neal",
 'Steph Curry',
 'Steve Nash',
 'Tim Duncan',
 'Tracy McGrady',
 'Vince Carter',
 'Wilt Chamberlain']

In [ ]:
top_players_overall[['Name', 'Position', 'Predicted_Rating', 'Rating', 'Diff']].

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = merged_df.copy()

# Drop rows with any NaN values
df_clean = df.dropna()
df = df_clean.copy()

# Assuming df is your DataFrame
X = df.drop(columns=['Name', 'Position', 'Points'])  # Features
y = df['Points']  # Target variable, assuming Rating is a composite score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the 'Rating' for each player
df['Predicted_Points'] = model.predict(X)

# Rank players based on predicted rating across the entire dataset
df['Rank'] = df['Predicted_Points'].rank(ascending=False)

# Now you can see the top players overall
top_players_overall = df.sort_values(by='Rank').head(50)
print("Top players overall:")
print(top_players_overall[['Name', 'Position', 'Predicted_Points', 'Points', 'Rank']])


Top players overall:
                        Name Position  Predicted_Points  Points  Rank
80              Kevin Durant       SF           38.7222   41.04   1.0
43               Kobe Bryant       SG           37.3862   39.12   2.0
167           Damian Lillard       PG           35.8292   35.83   3.0
119             James Harden       PG           35.6699   36.92   4.0
154           Michael Jordan       SG           35.3198   31.36   5.0
31                Larry Bird       SF           34.9449   34.16   6.0
62               Steph Curry       PG           34.1921   33.12   7.0
186            Tracy McGrady       SG           32.9245   32.44   8.0
8            Carmelo Anthony       SF           32.4369   32.96   9.0
190            Reggie Miller       SG           32.2389   32.83  10.0
7      Giannis Antetokounmpo       PF           31.8754   37.71  11.0
235          Oscar Robertson       PG           30.2712   29.68  12.0
0        Kareem Abdul-Jabbar        C           30.2449   29.68  13.0

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = merged_df.copy()

# Drop rows with any NaN values
df_clean = df.dropna()
df = df_clean.copy()

# Assuming df is your DataFrame
X = df.drop(columns=['Name', 'Position', 'Minutes'])  # Features
y = df['Minutes']  # Target variable, assuming Rating is a composite score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the 'Rating' for each player
df['Predicted_Minutes'] = model.predict(X)

# Rank players based on predicted rating across the entire dataset
df['Rank'] = df['Predicted_Minutes'].rank(ascending=False)

# Now you can see the top players overall
top_players_overall = df.sort_values(by='Rank').head(50)
print("Top players overall:")
print(top_players_overall[['Name', 'Position', 'Predicted_Minutes', 'Minutes', 'Rank']])


Top players overall:
                        Name Position  Predicted_Minutes  Minutes  Rank
31                Larry Bird       SF            41.8265    43.48   1.0
80              Kevin Durant       SF            40.7191    41.56   2.0
148            Magic Johnson       PG            39.8974    41.60   3.0
79                Tim Duncan       PF            39.8683    40.76   4.0
62               Steph Curry       PG            39.7886    40.20   5.0
0        Kareem Abdul-Jabbar        C            39.5578    39.52   6.0
236           David Robinson        C            39.4220    41.08   7.0
119             James Harden       PG            39.2279    39.67   8.0
164            Kawhi Leonard       SF            39.2100    40.08   9.0
270            Klay Thompson       SG            39.1588    40.84  10.0
75               Luka Doncic       PG            39.0381    39.76  11.0
43               Kobe Bryant       SG            38.9362    38.80  12.0
51          Wilt Chamberlain        C      